In [1]:
import pandas as pd
import requests
from sqlalchemy import create_engine


path = '/home/emmanuel/Desktop/descarga_db/raw_data_project_m1.db'
engine = create_engine(f'sqlite:///{path}')
raw_df = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", engine)


In [29]:
#personal info dataframe

In [2]:
#query to get the df information

df_personal = pd.read_sql('personal_info',engine)
df_country = pd.read_sql('country_info',engine)
df_career = pd.read_sql('career_info',engine)
df_poll = pd.read_sql('poll_info',engine)


In [3]:
print(df_personal.shape)
print(df_country.shape)
print(df_career.shape)
print(df_poll.shape)

(9649, 5)
(9649, 3)
(9649, 4)
(9649, 6)


In [4]:
engine.dispose()

In [5]:
df_personal.head(1)

,uuid,age,gender,dem_has_children,age_group
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,male,NO,40_65


In [6]:
def gender_modifier(x):
    if x == 'FeMale':
        return 'Female'
    elif x == 'Fem':
        return 'Female'
    elif x == 'female':
        return 'Female'
    elif x == 'male':
        return 'Male'
    elif x == 'Male':
        return 'Male'

In [7]:
df_personal['dem_has_children'].unique()

array(['NO', 'yES', 'nO', 'YES'], dtype=object)

In [8]:
def children_modifier(x):
    if x == 'NO':
        return 'no'
    elif x == 'nO':
        return 'no'
    elif x == 'No':
        return 'no'
    elif x == 'YES':
        return 'yes'
    elif x == 'yES':
        return 'yes'

In [9]:
#appliying my definition to clean gender and dem_has_children columns
df_personal['gender_modified'] = df_personal['gender'].apply(gender_modifier)
df_personal['dem_has_children_mo'] = df_personal['dem_has_children'].apply(children_modifier)

#from df raw, dropping the old and unclean 'gender' and dem_has_children columns
df_personal_clean = df_personal.drop(columns=['gender','dem_has_children'])


#rename column
df_personal_clean = df_personal_clean.rename(columns={'gender_modified':'gender','dem_has_children_mo':'dem_has_children'})

In [10]:
df_personal_clean.isnull().sum()#no nulls
df_personal_clean.head()

,uuid,age,age_group,gender,dem_has_children
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61 years old,40_65,Male,no
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57 years old,40_65,Male,yes
2,83127080-da3d-0133-c74f-0a81e8b09a82,32 years old,26_39,Male,no
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45 years old,40_65,Male,yes
4,24954a70-db98-0133-4a64-0a81e8b09a82,41 years old,40_65,Female,yes


In [18]:
df_personal_clean.to_csv('../raw_data/df_personal.csv')

In [22]:
#country info dataframe

In [31]:
df_country.isnull().sum()# no nulls
df_country.to_csv('../raw_data/df_country.csv')#sent them to raw data folder

In [11]:
#career info dataframe
df_career['dem_education_level'].unique()

array(['no', 'high', None, 'medium', 'low'], dtype=object)

In [12]:
df_career.isnull().sum()

uuid                      0
dem_education_level     663
dem_full_time_job         0
normalized_job_code    3947
dtype: int64

In [13]:
df_career['normalized_job_code'].fillna('None',inplace=True)
df_career['dem_education_level'].fillna('None',inplace=True)

In [12]:
df_career.to_csv('../raw_data/df_career.csv')

In [ ]:
#poll info dataframe

In [14]:
df_poll.isnull().sum()

uuid                                                   0
question_bbi_2016wave4_basicincome_awareness           0
question_bbi_2016wave4_basicincome_vote                0
question_bbi_2016wave4_basicincome_effect              0
question_bbi_2016wave4_basicincome_argumentsfor        0
question_bbi_2016wave4_basicincome_argumentsagainst    0
dtype: int64

In [16]:
df_poll.to_csv('../raw_data/df_poll.csv')

In [15]:
jobs_list = df_career['normalized_job_code'].unique().tolist()


In [16]:
jobs_dict = {}

for a in jobs_list:
    
    r = requests.get(f'http://api.dataatwork.org/v1/jobs/{a}')
    jobs_dict[a]=(r.json())['normalized_job_title']
          

KeyError: 'normalized_job_title'

In [17]:

def api_caller(x):
    r = requests.get('http://api.dataatwork.org/v1/jobs/' + str(x))
    req = r.json()
    return ["normalized_job_title"]

In [18]:
df_career['normalized_job_code'].apply(api_caller)

KeyboardInterrupt: 

In [72]:
#country codes

country_list = df_country['country_code'].unique().tolist()
country_dict = {}
country_list

['AT',
 'BE',
 'BG',
 'CY',
 'CZ',
 'DE',
 'DK',
 'EE',
 'ES',
 'FI',
 'FR',
 'GB',
 'GR',
 'HR',
 'HU',
 'IE',
 'IT',
 'LT',
 'LU',
 'LV',
 'MT',
 'NL',
 'PL',
 'PT',
 'RO',
 'SE',
 'SI',
 'SK']

In [61]:
for i in country_list:
    r= requests.get(f'https://restcountries.eu/rest/v2/alpha/{i}')
    country_dict[i]:r.json()['name']
        

In [58]:
pd.DataFrame(country_dict.items(),columns=['country_name','country_code']).to_csv('../raw_data/country_codes.csv')

In [94]:
#country codes through webscrapping

url = 'https://www.iban.com/country-codes' 
raw_codes = pd.read_html(url,header=0)[0]
raw_codes

#only keeping columns that are needed
country_codes = raw_codes.drop(columns=['Alpha-3 code','Numeric'])
country_code = country_codes.rename(columns={'Country':'country','Alpha-2 code':'country_code'})

#saving all the country codes
